# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill</span>

## 🗒️ This notebook is divided in 3 sections:
1. Load the data and process features.
2. Connect to the Hopsworks feature store.
3. Create feature groups and upload them to the feature store.

## <span style='color:#2656a3'> ⚙️ Import of libraries and packages

In [1]:
# Install of the packages for hopsworks
# !pip install -U hopsworks --quiet

In [1]:
# Importing of the packages for the needed libraries for the Jupyter notebook
import pandas as pd
import requests
import hopsworks
import os

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')

c:\Users\Benj3\anaconda3\envs\bds-streamlit\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## <span style="color:#2656a3;"> 💽 Load the historical data

The data you will use comes from three different sources:

- Electricity prices in Denmark per day from [Energinet](https://www.energidataservice.dk).
- Different meteorological observations from [Open meteo](https://www.open-meteo.com).

### <span style="color:#2656a3;">💸 Electricity prices per day from Energinet
*Hvis vi skal have tariffer med i modellen, anbefales det at vi sætter en faktor på 0.2 i tidsrummet 22 - 16 og en faktor på 0.6 eller 0.7 i tidsrummet 17 - 21.*

In [2]:
# Defining the URL for the API call to the electricity price data
electricity_api_url = ('https://api.energidataservice.dk/dataset/Elspotprices?offset=0&start=2022-01-01T00:00&end=2023-12-31T23:59&filter=%7B%22PriceArea%22:[%22DK1%22]%7D&sort=HourUTC%20DESC')

In [3]:
# Fetch data from the API and make the output to a pandas dataframe
electricity_data = requests.get(electricity_api_url).json()
electricity_df = pd.DataFrame(electricity_data['records'])

In [4]:
# Display the first 5 rows of the dataframe
electricity_df.head()

,HourUTC,HourDK,PriceArea,SpotPriceDKK,SpotPriceEUR
0,2023-12-31T22:00:00,2023-12-31T23:00:00,DK1,200.309998,26.870001
1,2023-12-31T21:00:00,2023-12-31T22:00:00,DK1,213.729996,28.670000
2,2023-12-31T20:00:00,2023-12-31T21:00:00,DK1,220.660004,29.600000
3,2023-12-31T19:00:00,2023-12-31T20:00:00,DK1,260.100006,34.889999
4,2023-12-31T18:00:00,2023-12-31T19:00:00,DK1,295.510010,39.639999


In [5]:
# Datapreprocessing by making the spotprice per kwh instead of mwh
electricity_df['SpotPriceDKK_KWH'] = electricity_df['SpotPriceDKK'] / 1000

In [6]:
# Datacleaning by removing the columns that are not needed
electricity_df.drop('SpotPriceDKK', axis=1, inplace=True)
electricity_df.drop('SpotPriceEUR', axis=1, inplace=True)
electricity_df.drop('HourUTC', axis=1, inplace=True)

In [7]:
# Renaming the columns and reformating the time column
electricity_df.rename(columns={'HourDK': 'time'}, inplace=True)
electricity_df['time'] = electricity_df['time'].astype(str).str[:-3]

In [8]:
electricity_df['date'] = electricity_df['time'].str[:10]

In [9]:
new_column = electricity_df.pop('date')
electricity_df.insert(0, 'date', new_column)

In [15]:
# Convert string 'date' column to DATE type
electricity_df['date'] = pd.to_datetime(electricity_df['date'], format='%Y-%m-%d').dt.date
 

In [16]:
# Display the first 5 rows of the dataframe
electricity_df.head()

,date,time,PriceArea,SpotPriceDKK_KWH
0,2023-12-31,2023-12-31T23:00,DK1,0.20031
1,2023-12-31,2023-12-31T22:00,DK1,0.21373
2,2023-12-31,2023-12-31T21:00,DK1,0.22066
3,2023-12-31,2023-12-31T20:00,DK1,0.26010
4,2023-12-31,2023-12-31T19:00,DK1,0.29551


### <span style="color:#2656a3;"> 🌤 Meteorological measurements from Open Meteo

Burde have enddate 2023-12-31. url = ("https://archive-api.open-meteo.com/v1/archive?latitude=57.048&longitude=9.9187&start_date=2022-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m")

In [17]:
# Defining the URL for the API call to the weather data   
weather_api_url = ("https://archive-api.open-meteo.com/v1/archive?latitude=57.048&longitude=9.9187&start_date=2022-01-01&end_date=2023-12-31&hourly=temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m")

In [18]:
# Fetch data from the API and make the output to a pandas dataframe
weather_data = requests.get(weather_api_url).json()
weather_df = pd.DataFrame(weather_data['hourly'])

In [19]:
weather_df['date'] = weather_df['time'].str[:10]

In [20]:
new_column = weather_df.pop('date')
weather_df.insert(0, 'date', new_column)

In [21]:
# Convert string 'date' column to DATE type
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%Y-%m-%d').dt.date

In [22]:
# Display the first 5 rows of the dataframe
weather_df.head()

,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,2022-01-01,2022-01-01T00:00,6.7,100,0.0,0.0,0.0,3,100,16.2,36.0
1,2022-01-01,2022-01-01T01:00,6.6,100,0.0,0.0,0.0,3,100,16.2,30.2
2,2022-01-01,2022-01-01T02:00,6.7,99,0.0,0.0,0.0,3,100,15.5,30.6
3,2022-01-01,2022-01-01T03:00,6.7,100,0.0,0.0,0.0,3,100,12.7,28.8
4,2022-01-01,2022-01-01T04:00,6.7,99,0.0,0.0,0.0,3,100,10.6,23.8


### Calendar of Danish workdays and holidays 

In [23]:
# Read csv file with calender
calender_df = pd.read_csv('Data/calendar_incl_holiday.csv', delimiter=';', usecols=['date', 'type'])
 
# Display the DataFrame
calender_df

,date,type
0,01/01/2022,Not a Workday
1,02/01/2022,Not a Workday
2,03/01/2022,Workday
3,04/01/2022,Workday
4,05/01/2022,Workday
...,...,...
1091,27/12/2024,Workday
1092,28/12/2024,Not a Workday
1093,29/12/2024,Not a Workday
1094,30/12/2024,Workday


In [24]:
calender_df['date'] = calender_df['date'].str.replace('/', '-')
calender_df

,date,type
0,01-01-2022,Not a Workday
1,02-01-2022,Not a Workday
2,03-01-2022,Workday
3,04-01-2022,Workday
4,05-01-2022,Workday
...,...,...
1091,27-12-2024,Workday
1092,28-12-2024,Not a Workday
1093,29-12-2024,Not a Workday
1094,30-12-2024,Workday


In [33]:
# flip the date to the left for the date column
def flip_dates_left_in_column(column):
    flipped_column = [flip_date_left(date) for date in column]
    return flipped_column

In [29]:
# flip the date to the left
def flip_date_left(date):
    parts = date.split("-")  # Assuming the date format is "YYYY-MM-DD"
    flipped_date = "-".join(parts[::-1])
    return flipped_date

In [30]:
flipped_dates_column = flip_dates_left_in_column(calender_df['date'])

In [31]:
calender_df['date'] = flipped_dates_column

In [35]:
# Convert string 'date' column to DATE type
calender_df['date'] = pd.to_datetime(calender_df['date'], format='%Y-%m-%d').dt.date
calender_df

,date,type
0,2022-01-01,Not a Workday
1,2022-01-02,Not a Workday
2,2022-01-03,Workday
3,2022-01-04,Workday
4,2022-01-05,Workday
...,...,...
1091,2024-12-27,Workday
1092,2024-12-28,Not a Workday
1093,2024-12-29,Not a Workday
1094,2024-12-30,Workday


## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

In [19]:
project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550040
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#2656a3;"> 🪄 Creating Feature Groups

In [20]:
# Creating the feature group for the weater data
weather_fg = fs.get_or_create_feature_group(
    name="weather_measurements",
    version=1,
    description="Weather measurements from Open Meteo API",
    primary_key=["date"],
    online_enabled=True,
    event_time="time",
)

In [21]:
# Inserting the weather_df into the feature group named weather_fg
weather_fg.insert(weather_df)

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/550040/featurestores/545863/featuregroups). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270171,"usrMsg":", the provided event time feature `time` is of type `string` but can only be one of the following types: [TIMESTAMP, DATE, BIGINT].","errorMsg":"Illegal event time feature type"}', error code: 270171, error msg: Illegal event time feature type, user msg: , the provided event time feature `time` is of type `string` but can only be one of the following types: [TIMESTAMP, DATE, BIGINT].

In [ ]:
# Creating the feature group for the electricity prices
electricity_fg = fs.get_or_create_feature_group(
    name="electricity_prices",
    version=1,
    description="Electricity prices from Energidata API",
    primary_key=["date"],
    online_enabled=True,
    event_time="time",
)

In [ ]:
# Inserting the electricity_df into the feature group named electricity_fg
electricity_fg.insert(electricity_df)

In [ ]:
danish_holidays_fg = fs.get_or_create_feature_group(
    name="danish_holidays",
    version=1,
    description="Danish holidays calendar.",
    online_enabled=True,
    primary_key=["date"],
)

In [ ]:
danish_holidays_fg.insert(calender_df)

---
## <span style="color:#2656a3;">⏭️ **Next:** Part 02: Feature Pipeline </span>

In the next notebook, you will be generating new data for the Feature Groups.